In [12]:
#Parameters of MC estimation:

import numpy as np
import math

S_0 = 100. #Initial stock price
r = 0.05 #Risk-free rate
sig = 0.2 #Stock price volatility
K = 100. #Strike price
lam = 0.04 #Intensity of departure process

T = 10. #Time to expiry
N = 2**10 #Number of sub-intervals of [0,T]
dt = T/float(N) #Length of each sub-interval

L = 150 #Initial level of barrier
alpha = -0.02 #Decay parameter for barriers L_t and K_t
tau = 9. #Parameter for barrier L_tau(t)

M = 1000 #Number of Monte Carlo samples

####################################################

MC = np.zeros(M)

slope = (L - K)/(T - tau)

N_0 = int(N*tau/T)

for m in range(0, M):
    
    Z = np.random.normal(0, 1, N)

    S = np.zeros(N)
    S[0] = S_0
     
    #Indicators:
    I = 0 #Departure
    J = 0 #Early exercise
    
    for i in range(0, N-1):
        
        S[i+1] = S[i]*np.exp( (r - 0.5*(sig**2))*dt + sig*np.sqrt(dt)*Z[i] )
         
    for j in range(0, N_0):
        
        P = np.random.poisson(lam*dt, 1)
            
        if P >= 1:
            I = 1
            t = (j+1)*dt
            S_t = S[j+1]
            break
            
        if S[j+1] > L:
#Change above line to if S[j+1] > L*np.exp(alpha*(i+1)*dt) for barrier L_t
#Change above line to if S[j+1] > K*np.exp(alpha*((i+1)*dt - T)) for barrier K_t
            J = 1
            t = (j+1)*dt
            S_t = S[j+1]
            break
            
    for k in range(N_0, N-1):
        
        P = np.random.poisson(lam*dt, 1)
            
        if P >= 1:
            I = 1
            t = (j+1)*dt
            S_t = S[j+1]
            break
            
        if S[j+1] > K + slope*(T - (i+1)*dt):
#Change above line to if S[j+1] > L*np.exp(alpha*(i+1)*dt) for barrier L_t
#Change above line to if S[j+1] > K*np.exp(alpha*((i+1)*dt - T)) for barrier K_t
            J = 1
            t = (j+1)*dt
            S_t = S[j+1]
            break
            
    if I == 1:
        V = np.exp(-r*t)*np.maximum(S_t - K, 0)
    elif J == 1:
        V = np.exp(-r*t)*np.maximum(S_t - K, 0)
    else:
        V = np.exp(-r*T)*np.maximum(S[N-1] - K, 0)

    MC[m] = V 


Am = round(np.mean(MC), 4)
Bm = round(np.std(MC), 4)

CI = [round(Am - 1.96*Bm/np.sqrt(M), 4), round(Am + 1.96*Bm/np.sqrt(M), 4)]  

#########################################################

print('Brute-force MC estimate:', Am, 'with confidence interval', CI)

Brute-force MC estimate: 26.0027 [24.7241, 27.2813]
